In [1]:
import numpy as np
import pyvista as pv
import trimesh as tm
from eikonax import preprocessing as ekx_preprocessing
from eikonax import solver as ekx_solver
from eikonax import tensorfield as ekx_tensorfield

In [14]:
patient_id = "06"
infile_name = f"patient_{patient_id}_mesh_with_fibers_tags.vtk"
anatomical_tags = {"Body": 0, "LAA": 1, "LIPV": 2, "LSPV": 3, "RIPV": 4, "RSPV": 5}

vtk_mesh = pv.read(infile_name)
vertices = vtk_mesh.points
simplices = vtk_mesh.cells.reshape(-1, 4)[:, 1:]
triangular_mesh = pv.PolyData.from_regular_faces(vertices, simplices)
tm_mesh = tm.Trimesh(vertices=vertices, faces=simplices, process=False)

laa_region = vtk_mesh.extract_values(anatomical_tags["LAA"], scalars="anatomical_tags")
lipv_region = vtk_mesh.extract_values(
    anatomical_tags["LIPV"], scalars="anatomical_tags"
)
lspv_region = vtk_mesh.extract_values(
    anatomical_tags["LSPV"], scalars="anatomical_tags"
)
ripv_region = vtk_mesh.extract_values(
    anatomical_tags["RIPV"], scalars="anatomical_tags"
)
rspv_region = vtk_mesh.extract_values(
    anatomical_tags["RSPV"], scalars="anatomical_tags"
)

mesh_boundaries = vtk_mesh.extract_feature_edges(
    boundary_edges=True,
    feature_edges=True,
    manifold_edges=False,
    non_manifold_edges=False,
)
laa_boundaries = laa_region.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)
lipv_boundaries = lipv_region.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)
lspv_boundaries = lspv_region.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)
ripv_boundaries = ripv_region.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)
rspv_boundaries = rspv_region.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)

In [15]:
mesh_boundary_points = mesh_boundaries.points.view(
    [("", mesh_boundaries.points.dtype)] * mesh_boundaries.points.shape[1]
)
laa_boundary_points = laa_boundaries.points.view(
    [("", laa_boundaries.points.dtype)] * laa_boundaries.points.shape[1]
)
lipv_boundary_points = lipv_boundaries.points.view(
    [("", lipv_boundaries.points.dtype)] * lipv_boundaries.points.shape[1]
)
lspv_boundary_points = lspv_boundaries.points.view(
    [("", lspv_boundaries.points.dtype)] * lspv_boundaries.points.shape[1]
)
ripv_boundary_points = ripv_boundaries.points.view(
    [("", ripv_boundaries.points.dtype)] * ripv_boundaries.points.shape[1]
)
rspv_boundary_points = rspv_boundaries.points.view(
    [("", rspv_boundaries.points.dtype)] * rspv_boundaries.points.shape[1]
)

local_laa_inner_boundary_inds = np.where(
    ~np.isin(laa_boundary_points, mesh_boundary_points)
)[0]
local_lipv_boundaries_inds = np.where(
    ~np.isin(lipv_boundary_points, mesh_boundary_points)
)[0]
local_lspv_boundaries_inds = np.where(
    ~np.isin(lspv_boundary_points, mesh_boundary_points)
)[0]
local_ripv_boundaries_inds = np.where(
    ~np.isin(ripv_boundary_points, mesh_boundary_points)
)[0]
local_rspv_boundaries_inds = np.where(
    ~np.isin(rspv_boundary_points, mesh_boundary_points)
)[0]

global_laa_inner_boundary_ids = laa_region.point_data["vtkOriginalPointIds"][
    laa_boundaries.point_data["vtkOriginalPointIds"][local_laa_inner_boundary_inds]
]
global_lipv_inner_boundary_ids = lipv_region.point_data["vtkOriginalPointIds"][
    lipv_boundaries.point_data["vtkOriginalPointIds"][local_lipv_boundaries_inds]
]
global_lspv_inner_boundary_ids = lspv_region.point_data["vtkOriginalPointIds"][
    lspv_boundaries.point_data["vtkOriginalPointIds"][local_lspv_boundaries_inds]
]
global_ripv_inner_boundary_ids = ripv_region.point_data["vtkOriginalPointIds"][
    ripv_boundaries.point_data["vtkOriginalPointIds"][local_ripv_boundaries_inds]
]
global_rspv_inner_boundary_ids = rspv_region.point_data["vtkOriginalPointIds"][
    rspv_boundaries.point_data["vtkOriginalPointIds"][local_rspv_boundaries_inds]
]

inner_boundary_tags = np.zeros(laa_boundaries.n_points)
inner_boundary_tags[local_laa_inner_boundary_inds] = 1
laa_boundaries.point_data["inner_boundary"] = inner_boundary_tags
laa_inner_boundary = laa_boundaries.extract_values(1, scalars="inner_boundary")

inner_boundary_tags = np.zeros(lipv_boundaries.n_points)
inner_boundary_tags[local_lipv_boundaries_inds] = 1
lipv_boundaries.point_data["inner_boundary"] = inner_boundary_tags
lipv_inner_boundary = lipv_boundaries.extract_values(1, scalars="inner_boundary")

inner_boundary_tags = np.zeros(lspv_boundaries.n_points)
inner_boundary_tags[local_lspv_boundaries_inds] = 1
lspv_boundaries.point_data["inner_boundary"] = inner_boundary_tags
lspv_inner_boundary = lspv_boundaries.extract_values(1, scalars="inner_boundary")

inner_boundary_tags = np.zeros(ripv_boundaries.n_points)
inner_boundary_tags[local_ripv_boundaries_inds] = 1
ripv_boundaries.point_data["inner_boundary"] = inner_boundary_tags
ripv_inner_boundary = ripv_boundaries.extract_values(1, scalars="inner_boundary")

inner_boundary_tags = np.zeros(rspv_boundaries.n_points)
inner_boundary_tags[local_rspv_boundaries_inds] = 1
rspv_boundaries.point_data["inner_boundary"] = inner_boundary_tags
rspv_inner_boundary = rspv_boundaries.extract_values(1, scalars="inner_boundary")

In [16]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    vtk_mesh,
    style="wireframe",
    color="lightgray",
)
plotter.add_mesh(
    laa_inner_boundary,
    color="blue",
    line_width=5,
    render_lines_as_tubes=True,
)
plotter.add_mesh(
    lipv_inner_boundary,
    color="green",
    line_width=5,
    render_lines_as_tubes=True,
)
plotter.add_mesh(
    lspv_inner_boundary,
    color="yellow",
    line_width=5,
    render_lines_as_tubes=True,
)
plotter.add_mesh(
    ripv_inner_boundary,
    color="purple",
    line_width=5,
    render_lines_as_tubes=True,
)
plotter.add_mesh(
    rspv_inner_boundary,
    color="orange",
    line_width=5,
    render_lines_as_tubes=True,
)
plotter.show()

Widget(value='<iframe src="http://localhost:40873/index.html?ui=P_0x7fcb83570910_4&reconnect=auto" class="pyvi…

In [5]:
mesh_data = ekx_preprocessing.MeshData(vertices, simplices)
solver_data = ekx_solver.SolverData(
    tolerance=1e-6,
    max_num_iterations=1000,
    max_value=100000,
    loop_type="jitted_while",
    use_soft_update=False,
    softminmax_order=100,
    softminmax_cutoff=0.01,
)
initial_sites = ekx_preprocessing.InitialSites(
    inds=global_lipv_inner_boundary_ids,
    values=np.zeros_like(global_lipv_inner_boundary_ids),
)
parameter_vector = np.ones(simplices.shape[0])

tensor_on_simplex = ekx_tensorfield.InvLinearScalarSimplexTensor(
    dimension=vertices.shape[1]
)
tensor_field_mapping = ekx_tensorfield.LinearScalarMap()
tensor_field_object = ekx_tensorfield.TensorField(
    num_simplices=simplices.shape[0],
    vector_to_simplices_map=tensor_field_mapping,
    simplex_tensor=tensor_on_simplex,
)
tensor_field_instance = tensor_field_object.assemble_field(parameter_vector)
eikonal_solver = ekx_solver.Solver(mesh_data, solver_data, initial_sites)
solution = eikonal_solver.run(tensor_field_instance)
solution_on_lspv_boundary = solution.values[global_lspv_inner_boundary_ids]

optimal_lspv_ind = global_lspv_inner_boundary_ids[
    int(np.argmin(solution_on_lspv_boundary))
]

initial_sites = ekx_preprocessing.InitialSites(
    inds=(optimal_lspv_ind,),
    values=(0,),
)
eikonal_solver = ekx_solver.Solver(mesh_data, solver_data, initial_sites)
solution = eikonal_solver.run(tensor_field_instance)
solution_on_lipv_boundary = solution.values[global_lipv_inner_boundary_ids]
optimal_lipv_ind = global_lipv_inner_boundary_ids[
    int(np.argmin(solution_on_lipv_boundary))
]

lipv_lspv_path = triangular_mesh.geodesic(
    start_vertex=optimal_lipv_ind,
    end_vertex=optimal_lspv_ind,
)

arc_lengths = [
    0,
]

for i, current_vertex_ind in enumerate(lipv_lspv_path.lines[2:]):
    last_vertx_ind = lipv_lspv_path.lines[i + 1]
    arc_lengths.append(
        arc_lengths[-1]
        + np.linalg.norm(
            lipv_lspv_path.points[current_vertex_ind]
            - lipv_lspv_path.points[last_vertx_ind]
        )
    )

arc_lengths = np.array(arc_lengths)
normalized_arc_lengths = arc_lengths / arc_lengths[-1]
lipv_lspv_midpoint = np.where(normalized_arc_lengths >= 0.5)[0][0]

In [6]:
initial_sites = ekx_preprocessing.InitialSites(
    inds=global_ripv_inner_boundary_ids,
    values=np.zeros_like(global_ripv_inner_boundary_ids),
)
eikonal_solver = ekx_solver.Solver(mesh_data, solver_data, initial_sites)
solution = eikonal_solver.run(tensor_field_instance)
solution_on_rspv_boundary = solution.values[global_rspv_inner_boundary_ids]

optimal_rspv_ind = global_rspv_inner_boundary_ids[
    int(np.argmin(solution_on_rspv_boundary))
]

initial_sites = ekx_preprocessing.InitialSites(
    inds=(optimal_rspv_ind,),
    values=(0,),
)
eikonal_solver = ekx_solver.Solver(mesh_data, solver_data, initial_sites)
solution = eikonal_solver.run(tensor_field_instance)
solution_on_ripv_boundary = solution.values[global_ripv_inner_boundary_ids]
optimal_ripv_ind = global_ripv_inner_boundary_ids[
    int(np.argmin(solution_on_ripv_boundary))
]

ripv_rspv_path = triangular_mesh.geodesic(
    start_vertex=optimal_ripv_ind,
    end_vertex=optimal_rspv_ind,
)

arc_lengths = [
    0,
]

for i, current_vertex_ind in enumerate(ripv_rspv_path.lines[2:]):
    last_vertx_ind = ripv_rspv_path.lines[i + 1]
    arc_lengths.append(
        arc_lengths[-1]
        + np.linalg.norm(
            ripv_rspv_path.points[current_vertex_ind]
            - ripv_rspv_path.points[last_vertx_ind]
        )
    )

arc_lengths = np.array(arc_lengths)
normalized_arc_lengths = arc_lengths / arc_lengths[-1]
ripv_rspv_midpoint = np.where(normalized_arc_lengths >= 0.5)[0][0]

In [7]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(vtk_mesh, scalars="anatomical_tags", show_edges=False)
plotter.add_points(
    lipv_lspv_path.points[0],
    color="orange",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_points(
    lipv_lspv_path.points[-1],
    color="blue",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_points(
    lipv_lspv_path.points[lipv_lspv_midpoint],
    color="green",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_mesh(lipv_lspv_path, color="red", line_width=5, render_lines_as_tubes=True)
plotter.add_points(
    ripv_rspv_path.points[0],
    color="orange",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_points(
    ripv_rspv_path.points[-1],
    color="blue",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_points(
    ripv_rspv_path.points[ripv_rspv_midpoint],
    color="green",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.add_mesh(ripv_rspv_path, color="red", line_width=5, render_lines_as_tubes=True)
plotter.show()

Widget(value='<iframe src="http://localhost:40873/index.html?ui=P_0x7fcb83570f50_1&reconnect=auto" class="pyvi…